In [2]:
# coding=utf-8
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import tensorflow as tf
from logging_future import logger

In [3]:
location = "C:/Users/workspace/brad_public_workspace_on_win/SH_tongliang/data/数据库/火币日线/" 
file = "11_14_huobi_btcusdt_copy_for_ML_.csv"
df = pd.read_csv(location + file, engine="python")

data_len = len(df)
total_days = data_len
test_end = total_days
train_begin = 0

# 参数类别一：
"""
保持以下参数不变：

利用过去一天: 2018-11-16 0:00, 来预测下一天：
time_step = 20
rnn_unit = 10  # 隐藏层单元数量 ##################调整这个
input_size = 3  # 输入个数  
output_size = 1  # 输出个数
batch_size = 80  # 批量大小  #################调整这个
train_times = 20  # 训练次数
lr = 0.001  # 学习率


test_train_split 参数调优:

test_train_split   accuracy       信号强度
       0.2           0.55         -55.86
       0.3           0.52         -39.31
       0.4           0.4928        -9.88
       0.5           0.5111       -28.13
       0.6           0.5          -13.87
       0.7           0.4961       -27.13
       0.8           0.4967       -28.58
       0.9           0.5          -9.06

"""

test_train_split = 0.5
time_step = 20
split_batch = int(total_days/time_step) 
time_step_multiple = int(test_train_split*split_batch)
test_begin = int(test_end - (time_step)*time_step_multiple)
train_end = test_begin

# 这里 test_end - test_begin的差一定要是time_step的倍数
print ("split_batch: ", split_batch)
print ("train_test_split_rate: ", test_train_split)
print ("train_begin: ", train_begin)
print ("train_end: ", train_end)
print ("test_begin: ", test_begin)
print ("test_end: ", test_end)
print ("time_step: ", time_step)

split_batch:  19
train_test_split_rate:  0.5
train_begin:  0
train_end:  207
test_begin:  207
test_end:  387
time_step:  20


In [4]:
# 参数类别二：

# Use this code to prevent the kernel from requesting to start again everytime we run the model. 
tf.reset_default_graph()

rnn_unit = 10  # 隐藏层单元数量 ##################调整这个
input_size = 3  # 输入个数  
output_size = 1  # 输出个数
batch_size = 80  # 批量大小  #################调整这个
train_times = 20  # 训练次数
lr = 0.001  # 学习率
# 一般调整隐藏层数量、批量大小及学习率这几个超参数
# 输入和输出则由特征量和标签确定
# 本例中，以开盘价、最高价、最低价为特征量，
# 以收盘价差即涨跌作为标签


pre_data = df.iloc[:, 4].values  # 取收盘价计算标签
label = []
for i in range(1, len(pre_data)):
    label.append(round(pre_data[i] - pre_data[i - 1], 4))

df.loc[1:, 'label'] = label
df['label'] = df['label'].shift(-1)
df.fillna(method='ffill', inplace = True)
data = df.iloc[:, [1, 2, 3, 5]].values  # 获取特征量及标签，类型为np.ndarray
# logger.info(data[0:5])


def get_train_data(batch_size, time_step, train_begin, train_end):
    batch_index = []
    data_train = data[train_begin + 1:train_end]
    normalized_train_data = (
        data_train - np.mean(data_train, axis=0)) / np.std(
            data_train, axis=0)
    # logger.info('normalized_train_data.shape: ', normalized_train_data.shape)
    # logger.info(len(normalized_train_data))
    # logger.info(type(normalized_train_data))
    # logger.info(len([0, 2]))
    # logger.info(normalized_train_data[0:20])
    # logger.info(normalized_train_data[0:20, 1])

    train_x, train_y = [], []
    for i in range(len(normalized_train_data) - time_step):
        if i % batch_size == 0:
            batch_index.append(i)

        x = normalized_train_data[i:i + time_step, :3]
        y = normalized_train_data[i:i + time_step, 3, np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())

    batch_index.append(len(normalized_train_data) - time_step)
    return batch_index, train_x, train_y


def get_test_data(time_step, test_begin, test_end):
    data_test = data[test_begin:test_end]
    test_y = data_test[:, 3]
    mean = np.mean(data_test, axis=0)
    std = np.std(data_test, axis=0)
    normalized_test_data = (data_test - mean) / std
    size = (len(normalized_test_data) + time_step - 1) // time_step
    test_x, test_y = [], []
    for i in range(size - 1):
        x = normalized_test_data[i * time_step:(i + 1) * time_step, :3]
        y = normalized_test_data[i * time_step:(i + 1) * time_step, 3]
        test_x.append(x.tolist())
        test_y.extend(y)
        # print('type(y): ', type(y))

    test_x.append((normalized_test_data[(i + 1) * time_step:, :3]).tolist())
    test_y.extend((normalized_test_data[(i + 1) * time_step:, 3]).tolist())
    return mean, std, test_x, test_y


weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[
        rnn_unit,
    ])),
    'out': tf.Variable(tf.constant(0.1, shape=[
        1,
    ]))
}


# 定义神经网络变量
def lstm(X):
    batch_size = tf.shape(X)[0]
    time_step = tf.shape(X)[1]
    w_in = weights['in']
    b_in = biases['in']
    input_ = tf.reshape(X, [-1, input_size])
    # print('input_.shape: ', input_.shape)
    input_rnn = tf.matmul(input_, w_in) + b_in
    # print('input_rnn.shape: ', input_rnn.shape)
    # tensor转换成3维，作为cell的输入
    input_rnn = tf.reshape(input_rnn, [-1, time_step, rnn_unit])
    # print('input_rnn.shape: ', input_rnn.shape)
    cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    output_rnn, final_state = tf.nn.dynamic_rnn(
        cell, input_rnn, initial_state=init_state, dtype=tf.float32)
    output = tf.reshape(output_rnn, [-1, rnn_unit])
    w_out = weights['out']
    b_out = biases['out']
    pred = tf.matmul(output, w_out) + b_out
    return pred, final_state


# 训练模型
def train_lstm(batch_size, time_step, train_begin, train_end):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    Y = tf.placeholder(tf.float32, shape=[None, time_step, output_size])
    batch_index, train_x, train_y = get_train_data(batch_size, time_step,
                                                   train_begin, train_end)
    with tf.variable_scope('future_lstm'):
        pred, _ = lstm(X)
        loss = tf.reduce_mean(
            tf.square(tf.reshape(pred, [-1]) - tf.reshape(Y, [-1])))
        train_operation = tf.train.AdamOptimizer(lr).minimize(loss)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(train_times):
                for step in range(len(batch_index) - 1):
                    _, loss_ = sess.run(
                        [train_operation, loss],
                        feed_dict={
                            X:
                            train_x[batch_index[step]:batch_index[step + 1]],
                            Y: train_y[batch_index[step]:batch_index[step + 1]]
                        })
                # print('Number of iterations: {} , loss: {}'.format(i, loss_))
                logger.info('Number of iterations: {} , loss: {}'.format(
                    i, loss_))
#             print('model saved: ', saver.save(sess, 'model_save1/model.ckpt'))
            print('The train has finished')


train_lstm(batch_size, time_step, train_begin, train_end)


def prediction(time_step):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
#     print ("X: ", X)  #brad's peek
    mean, std, test_x, test_y = get_test_data(time_step, test_begin, test_end)
    with tf.variable_scope('future_lstm', reuse=True):
        pred, _ = lstm(X)
        saver = tf.train.Saver(tf.global_variables())
        with tf.Session() as sess:
            model_file = tf.train.latest_checkpoint('model_save1')
            saver.restore(sess, model_file)
            test_predict = []
            # brad's peek:
#             print("test_x")
#             print (test_x)
            for step in range(len(test_x)):
                # 一次出time_step个结果，results.shape: (20, 1)
                results = sess.run(pred, feed_dict={X: [test_x[step]]})
                # predict = tf.reshape(results, [-1])
                predict = results.reshape(-1)
                test_predict.extend(predict)
            # brad's peek:
#             print (test_predict)
            # 还原真实值
            # brad's peek:
#             print ("test_y",test_y)
            test_y = np.array(test_y) * std[3] + mean[3]
            test_predict = np.array(test_predict) * std[3] + mean[3]
            true_y = test_y
            
            
            # print('test_predict.shape: ', test_predict.shape)
            # print('true_y.shape: ', true_y.shape)
            out = np.c_[test_predict, true_y]
            out_csv = pd.DataFrame(
                data=out, index=None, columns=['prediction', 'true'])
            out_csv.to_csv('pred_and_true_20180623.csv')
            # 计算精度
            right, wrong = 0, 0
            calc_data = map(lambda x, y: tuple((x, y)), test_predict,
                            test_y[:len(test_predict)])
            for data in calc_data:
                if data[0] * data[1] > 0 or data[0] == data[1]:
                    right += 1
                else:
                    wrong += 1
            accuracy = right / (right + wrong)
            print("利用过去一天: "+str(df[:test_end].iloc[-1, :].values[0]) + ", 来预测下一天：")
            print (out_csv.iloc[-1, 0])
            logger.info('The accuracy of this prediction: ')
            logger.info(accuracy)
            # print('The accuracy of this prediction: ', accuracy)
            # 计算偏差程度
            # deviation = np.average(
            #     np.abs(test_predict - test_y[:len(test_predict)]) /
            #     test_y[:len(test_predict)])
            # print('The deviation of this prediction: ', deviation)
            # logger.info('The deviation of this prediction: ')
            # logger.info(deviation)
            # logger.info(np.abs(test_predict - test_y[:len(test_predict)]))
            # logger.info(test_y[:len(test_predict)])

#             fig = plt.figure()
#             fig.set_size_inches(64, 48)
#             ax = plt.gca()
#             ax.spines['left'].set_linewidth(3)
#             ax.spines['bottom'].set_linewidth(3)
#             # 设置刻度大小
#             rc('xtick', labelsize=40)
#             rc('ytick', labelsize=40)
#             plt.plot(
#                 list(range(len(test_predict))),
#                 test_predict,
#                 color='blue',
#                 label='predict',
#             )
#             plt.plot(
#                 list(range(len(test_y))), test_y, color='red', label='real')
#             # 设置图例及X、Y轴标签，label名称支持Tex语法
#             plt.legend(loc='best', fontsize=60)
#             plt.xlabel(r'$\rm{dailydata}$', fontdict={'size': 60})
#             plt.ylabel(r'$\rm{fluctuation}$', fontdict={'size': 60})
#             plt.savefig(
#                 'future_lstm_train{}_accuracy{:.4f}.png'.format(
#                     train_times, accuracy),
#                 dpi=300)
#             plt.show()

prediction(time_step)

2018-11-18 19:39:30,020 INFO in line 141: Number of iterations: 0 , loss: 0.158553808927536
2018-11-18 19:39:30,051 INFO in line 141: Number of iterations: 1 , loss: 0.14628973603248596
2018-11-18 19:39:30,079 INFO in line 141: Number of iterations: 2 , loss: 0.14612358808517456
2018-11-18 19:39:30,112 INFO in line 141: Number of iterations: 3 , loss: 0.1471441239118576
2018-11-18 19:39:30,141 INFO in line 141: Number of iterations: 4 , loss: 0.14589720964431763
2018-11-18 19:39:30,169 INFO in line 141: Number of iterations: 5 , loss: 0.1443776935338974
2018-11-18 19:39:30,199 INFO in line 141: Number of iterations: 6 , loss: 0.14406119287014008
2018-11-18 19:39:30,227 INFO in line 141: Number of iterations: 7 , loss: 0.14443384110927582
2018-11-18 19:39:30,256 INFO in line 141: Number of iterations: 8 , loss: 0.14440885186195374
2018-11-18 19:39:30,287 INFO in line 141: Number of iterations: 9 , loss: 0.14373034238815308
2018-11-18 19:39:30,316 INFO in line 141: Number of iterations: 

In [55]:
# 分析强度信号：
signal_loc = "C:/Users/workspace/brad_public_workspace_on_win/SH_tongliang/ipython_notebooks/"
signal_file = pd.read_csv(signal_loc + "pred_and_true_20180623.csv")
del signal_file['Unnamed: 0']

In [59]:
signal_file.head(10)

,prediction,label
0,-49.677948,8.23
1,-73.595337,-169.51
2,-113.652702,-536.17
3,-93.984497,-128.67
4,-56.462330,-62.93
5,-40.813553,32.63
6,-67.715805,-188.89
7,-29.459488,-113.38
8,-38.488152,196.57
9,-8.686329,-95.30


In [46]:
# signal_file.set_index('prediction', inplace = True)

In [57]:
signal_file.rename(columns={"true": "label"}, inplace = True)

In [58]:
df_to_merge = df[test_begin:].copy()
df_to_merge.head(10)

,time,open,high,low,close,label
207,2018-5-21 0:00,8391.99,8588.00,8343.83,8400.22,8.23
208,2018-5-22 0:00,8400.23,8433.20,8137.04,8230.71,-169.51
209,2018-5-23 0:00,8229.95,8258.41,7650.38,7694.54,-536.17
210,2018-5-24 0:00,7690.74,7770.00,7265.00,7565.87,-128.67
211,2018-5-25 0:00,7568.00,7654.31,7327.00,7502.94,-62.93
212,2018-5-26 0:00,7502.28,7618.00,7352.00,7535.57,32.63
213,2018-5-27 0:00,7535.57,7600.00,7238.46,7346.68,-188.89
214,2018-5-28 0:00,7346.95,7431.12,7159.50,7233.30,-113.38
215,2018-5-29 0:00,7233.89,7481.43,7038.08,7429.87,196.57
216,2018-5-30 0:00,7429.82,7564.66,7303.11,7334.57,-95.30


In [72]:
signal_file['time'] = df_to_merge['time'].values

In [73]:
signal_file

,prediction,label,time
0,-49.677948,8.23,2018-5-21 0:00
1,-73.595337,-169.51,2018-5-22 0:00
2,-113.652702,-536.17,2018-5-23 0:00
3,-93.984497,-128.67,2018-5-24 0:00
4,-56.462330,-62.93,2018-5-25 0:00
5,-40.813553,32.63,2018-5-26 0:00
6,-67.715805,-188.89,2018-5-27 0:00
7,-29.459488,-113.38,2018-5-28 0:00
8,-38.488152,196.57,2018-5-29 0:00
9,-8.686329,-95.30,2018-5-30 0:00


In [74]:
merged = signal_file.merge(df_to_merge, on = 'time')

In [78]:
del merged['label_x']
del merged['label_y']

In [79]:
merged = merged[['time', 'open', 'high', 'low', 'close', 'prediction']]

In [80]:
merged

,time,open,high,low,close,prediction
0,2018-5-21 0:00,8391.99,8588.00,8343.83,8400.22,-49.677948
1,2018-5-22 0:00,8400.23,8433.20,8137.04,8230.71,-73.595337
2,2018-5-23 0:00,8229.95,8258.41,7650.38,7694.54,-113.652702
3,2018-5-24 0:00,7690.74,7770.00,7265.00,7565.87,-93.984497
4,2018-5-25 0:00,7568.00,7654.31,7327.00,7502.94,-56.462330
5,2018-5-26 0:00,7502.28,7618.00,7352.00,7535.57,-40.813553
6,2018-5-27 0:00,7535.57,7600.00,7238.46,7346.68,-67.715805
7,2018-5-28 0:00,7346.95,7431.12,7159.50,7233.30,-29.459488
8,2018-5-29 0:00,7233.89,7481.43,7038.08,7429.87,-38.488152
9,2018-5-30 0:00,7429.82,7564.66,7303.11,7334.57,-8.686329


In [82]:
merged.to_csv(location+"BTC_ML_result_file_to_test_on_TB.csv", index = False)